In [1]:
import re
import os
import operator

In [2]:
def rgxer(sss):
    result = ''
    for c in sss:
        if c in '.<>*[]{}+':
            result += '[' + c + ']'
        else:
            result += c
    return result

def match(regex, token):
    return bool(re.match('^{}$'.format(regex), token))

capture_patterns = {
    'table' : '[a-zA-Z0-9_-]+',    
    'n' : '[0-9]+(\\.)?[0-9]+',
    'd' : '[0-9]+',
    'f' : '[0-9]+[.][0-9]+',
    's'  :'.*',
}

special_patterns = {
    '\\[([,;]) ?(%[a-zA-Z])+\\]' : '\\2(\\1\\2)*'
    
}

def to_number(num):
    if match(capture_patterns['f'], num):
        return float(num)
    elif match(capture_patterns['d'], num):
        return int(num)
    else:
        return num


In [3]:
class Node:
    """
    A simple Node implementation with Node children and attributes.
    """
    def __init__(self, name):
        self.name = name
        self.attributes = {}
        self.children = []
        
    def attribute(self, k, v):
        self.attributes[k] = v
        
    def get(self, k):
        return self.attributes.get(k, None)
        
    def child(self, node):
        """
        Adopt a child node.
        You can also pass just text
        """
        if type(node) == str and node:
            self.child(Node(node))
        else:
            self.children.append(node)
    
    def print(self, depth=0):
        """
        Pretty print the node and its children recursively
        with tabulations.
        """
        print(' ' * depth * 4 + self.name + ':')
        for c in self.children:
            c.print(depth + 1)
            
    def compute(self, actions, idents={}):
        """
        This function is used to compute the value of an expression.
        We assume it's already been reduced.
        """
        sentinel = object()
        
        # check if ident
        ident = idents.get(self.name, sentinel)
        if ident is not sentinel:
            print('IDENT', self.name, '>', ident)
            return ident
        
        # check if number
        nn = to_number(self.name)
        if type(nn) != str:
            return nn
        
        # check if existing action
        action = actions.get(self.name, None)
        if action:
            r = action(*[child.compute(actions, idents) for child in self.children])
            return r
        
        # else return raw string/object
        print('raw string: ' + self.name)
        return self.name
            
    def __repr__(self):
        return self.__str__()
            
    def __str__(self):
        return '<node({}) {}>'.format(self.name, len(self.children))

class Tree:
    """
    A token Tree class to hold a root Node.
    """
    def __init__(self, root=None):
        self.root = root

In [4]:
def split_in_two(ss, i, v=0):
    """
    Split a string in two given a splitting index.
    """
    return (ss[:i], ss[i+1 if not v else v:])

def find_all(ss, s):
    """
    Return all occurences of a substring found in a string.
    Ouput is (<start>, <end>) of occurence
    """
    pos = []
    p = 0
    offset = 0
    while p + 1 and ss:
        p = ss.find(s)
        if p + 1:
            pos.append((offset + p, offset + p + len(s)))
            ss = ss[p+1:]
            offset += p + 1
    return pos

# Our tokenization function. It should :
# be able to differentiate specific entities from common words (operators)
# but if those entities are all letters, we need to find a way to differentiate them from the rest of the words
# (they need a space)
# still need to parse "strings" and 'strings' or only one of the two
def tokenize(expr, entities=[]):
    """
    The big, fluffy tokenize function.
    This function will tokenize as following :
    You can give it a list of entities that will be prioritized over any token, 
    except if they are in a "string literal".
    """
    items = []
    item = ''
    entities = sorted(entities, key=len)
    entity_occs = [(entity, find_all(expr, entity)) for entity in entities]
    entity_occs = [e for e in entity_occs if e[1]]
    found_entities = {occ[0]: (occ[1], occs[0]) for occs in entity_occs for occ in occs[1]}
    
    ignoring = 0
    in_quotes = ''
    for i, e in enumerate(expr):
        if ignoring:
            ignoring -= 1
            continue
            
        if not in_quotes and e in ['"', "'"]:
            in_quotes = e
            items.append(item) if item else None
            item = e        
            
        elif in_quotes:
            if e == in_quotes:
                in_quotes = ''
                item += e
                items.append(item) if item else None
                item = ''
            else:
                item += e
        
        elif i in found_entities and not match('[a-zA-Z][a-zA-Z0-9]*', found_entities[i][1]):

            # if found entity is all letters and item is not empty, do nothing
            # else
            ignoring = found_entities[i][0] - i - 1
            items.append(item) if item else None
            item = ''
            items.append(found_entities[i][1])
        
        elif e in 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.-_':
            item += e
        elif e == ' ':
            items.append(item) if item else None
            item = ''
        else:
            items.append(item) if item else None
            item = ''
            items.append(e)
        
    if item:
        items.append(item)    
    
    return items

def build(expr, ops):
    """
    Builds an expression tree from a given string literal.
    This function only builds a tree following collections (parenthese, brackets).
    Next step is reducing following operators.
    We still need to pass operators to this function for them to be tokenized properly.
    """
    root = Node('expr')
    parents = []
    node = root
    for item in tokenize(expr, ops):
        if item in '({[':
            parents.append(node)
            c = Node(item)
            node.child(c)
            node = c
        elif item in ')}]':
            node = parents.pop()
        else:
            node.child(item)
    return root
                
def reduce(node, ops):
    """
    Reduces an expression tree given operators with varying levels of priority.
    For every node, the children are sorted by operator priority (or given a default
    score if they are literals, but such that they end up at the bottom of the sorted
    list). The default behavior here is that of binary operators : taking into account
    the node with the highest level of priority (if it's an operator), the rest of the
    list is split in two, and two temporary nodes are created to hold them. We then
    reduce those nodes recursively.
    Unary operators are not supported as of now, but I should add them at some point.
    """
    if not node or not node.children:
        return node
    
    if len(node.children) == 1:
        if node.children[0].name == '(':
            return reduce(node.children[0], ops)
        else:
            return node.children[0]
    
    rank = lambda child_ind : ops[node.children[child_ind].name] if node.children[child_ind].name in ops else max(ops.values()) + 1
    
    ranks = [(i, rank(i)) for i in range(len(node.children))]
    
    reduce_order, ranks = list(zip(*ranks))
    
    reduce_order = sorted(list(range(len(reduce_order))), 
                          key=rank)
    
    if(len(set(ranks))) == 1:
        return node
    
    split_node = Node(node.children[reduce_order[0]].name)
    reduced_tuple = split_in_two(node.children, reduce_order[0])
    
    if not reduced_tuple[0]:
        raise Exception('Error : binary operator {} expected left operand, right={}'.format(split_node.name, reduced_tuple[1]))
    elif not reduced_tuple[1]:
        raise Exception('Error : binary operator {} expected right operand, left={}'.format(split_node.name, reduced_tuple[0]))
        
    left_node = Node('$r')
    left_node.children = reduced_tuple[0]
    right_node = Node('$r') 
    right_node.children = reduced_tuple[1]
    reduced_left_node = reduce(left_node, ops)
    reduced_right_node = reduce(right_node, ops)
    split_node.children = [reduced_left_node, reduced_right_node]
    node.children = [split_node]
    return split_node
    

In [8]:
class ExpressionParser:
    """
    This class encapsulates the process of expression parsing.
    In a expression, you want to detect identifiers and operators,
    and parse the rest with custom operations. This class allows you
    to do it easily.
    
    Integrating variables into this project would require to handle context.
    One day, maybe ...
    """
    def __init__(self):
        pass
        self.idents = {}
        self.operators = {}
        self.actions = {}
    
    def ident(self, name, value=None):
        """
        Register an identifier in the parser. For example,
        if you want 'Yes' to have the value true, you must
        specify it here. Otherwise, it will be parsed as 
        a literal.
        You can consult an identifier by only providing a name.
        """
        if value is not None:
            self.idents[name] = value
        else:
            return self.idents.get(name, None)
        
    def op(self, name, level=-1, func=None, unary=False):
        """
        Register an operator for the parser. An operator has
        a level of priority, which will be used when reducing
        the expression tree. A lower level operator will be
        reduced first. (or last ? you get my point).
        For example, in a regular arithmetic expression, the order
        of priority when reducing would be :
        1 : +, - (equal priority)
        2 : /, * (equal priority, more or less)
        because you always multiply before adding, and so on.
        Actually the division is better done after the multiplication.
        You can specify a function for your operator. Default behavior is
        binary, as in the function will receive two parameters.
        For example, if my function is a simple addition, then I can use
        the addition operator provided by Python, operator.add.
        Unary support should come at some point.
        You can also use this function to inspect already existing operators
        just by providing a name.
        """
        if level < 0 and not func:
            return (name, self.operators.get(name, None), self.actions[name]) if name in self.operators else None
        else:
            self.operators[name] = level
            self.actions[name] = func
            
    def parse(self, expr):
        tree = build(expr, self.operators)
        rtree = reduce(tree, self.operators)
        return rtree
        
    def process(self, expr):
        rtree = self.parse(expr)
        if rtree:
            return rtree.compute(self.actions, self.idents)
        

## Arithmetic Expression

I haven't implemented unary operators yet though.

In [9]:

# unary left or right ?
        
myep = ExpressionParser()
myep.op('+', 1, func=operator.add)
myep.op('-', 1, func=operator.sub)
myep.op('*', 2, func=operator.mul)
myep.op('/', 2.5, func=operator.truediv)

test_dataset = [
    ('1 + 1', 2),
    ('3 * (3 * 4)', 3 * (3 * 4)),
    ('(3 * (3 + 254)) / (2 + 3 * 2)', (3 * (3 + 254)) / (2 + 3 * 2)),
    ('(((((((((((((12))))))) + 3)) * 5))))', (((((((((((((12))))))) + 3)) * 5))))),
    ('-2 + 3', 1)
]

print('%40s %20s %20s %10s' % ('Expression', 'Expected', 'Obtained', 'Result'))
print('-' * 93)
for t in test_dataset:
    res = None
    try:
        res = myep.process(t[0])
    except:
        pass
    print('%40s %20s %20s %10s' % (t[0], t[1], res, res == t[1]))

                              Expression             Expected             Obtained     Result
---------------------------------------------------------------------------------------------
                                   1 + 1                    2                    2       True
                             3 * (3 * 4)                   36                   36       True
           (3 * (3 + 254)) / (2 + 3 * 2)               96.375               96.375       True
    (((((((((((((12))))))) + 3)) * 5))))                   75                   75       True
                                  -2 + 3                    1                 None      False


## Logical expression

In [10]:
logicep = ExpressionParser()
logicep.op('||', 1, func=operator.or_)
logicep.op('&&', 2, func=operator.and_)
logicep.op('&', 2, func=operator.and_)
logicep.ident('true', True)
logicep.ident('false', False)

logicep.process('true && (false || false) & true')

IDENT true > True
IDENT false > False
IDENT false > False
IDENT true > True


False

## Mixed expression

In [11]:
qev = ExpressionParser()
qev.op('OR', 1, func=operator.or_)
qev.op('AND', 2, func=operator.and_)
qev.op('<', 3, func=operator.lt)
qev.op('>', 3, func=operator.gt)
qev.op('<=', 3, func=operator.le)
qev.op('>=', 3, func=operator.ge)
qev.op('=', 3, func=operator.eq)
qev.op('+', 4, func=operator.add)
qev.op('-', 4, func=operator.sub)
qev.op('/', 4.5, func=operator.truediv)
qev.op('*', 5, func=operator.mul)

expr = '2 + 3 = 5 OR 3 = 4'

qev.parse(expr).print()
qev.process(expr)

expr = 'client.id = rosbif.fid OR client.name = "bae"'

qev.parse(expr).print()

OR:
    =:
        +:
            2:
            3:
        5:
    =:
        3:
        4:
OR:
    =:
        client.id:
        rosbif.fid:
    =:
        client.name:
        "bae":
